In [2]:
from nltk.corpus import stopwords

with open("english_word.csv") as word_file:
    english_words = set(word.strip().lower() for word in word_file)

stopword=list(stopwords.words("my_stopwords"))


In [ ]:
import csv
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

rows = [] 
with open("Twitter_Data.csv", 'r',encoding="utf8") as csvfile: #contains raw data.
    csvreader = csv.reader(csvfile,delimiter=",") 
    fields = next(csvreader)
    for row in csvreader:    #tokenize the sentences.                                   
        #print(row[0])
        s=row[0]
        filtered_sentence=[]
        for w in s.split(" "): 
            if (w not in stopword) and (w in english_words):  #Lemmatize the words.
                word1 = wordnet_lemmatizer.lemmatize(w, pos = "n")
                word2 = wordnet_lemmatizer.lemmatize(word1, pos = "v")
                word3 = wordnet_lemmatizer.lemmatize(word2, pos = "a")
                filtered_sentence.append(word3) 
        s=' '.join(filtered_sentence)
        #print(s)
        if s:
            rows.append([s,row[1]])     #stores the result in "rows".
#rows

In [4]:
with open("preprocessed_Data.csv", 'w',newline='') as csvfile:    #Append "rows" Pre-Processed data in new file.  
    csvwriter = csv.writer(csvfile) 
    csvwriter.writerow(['tweets', 'polarity']) 
    csvwriter.writerows(rows)

In [1]:
from sklearn.model_selection import train_test_split 
from sklearn import metrics
import pandas as pd

In [3]:
data = pd.read_csv("preprocessed_Data.csv")
data.isnull().sum()

tweets      0
polarity    0
dtype: int64

In [4]:
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

cv=TfidfVectorizer(ngram_range=(1,1))
text_count=cv.fit_transform(data["tweets"])

In [4]:
#cv.get_feature_names()[0:100]

In [5]:
text_count.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(text_count, data["polarity"], test_size=0.35, random_state=42) 

In [7]:
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.naive_bayes import ComplementNB
from sklearn.svm import LinearSVC

CNB=LinearSVC()
CNB.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [8]:
predicted=CNB.predict(X_test)
accuracy_Score=metrics.accuracy_score(predicted,y_test)

In [9]:
print(accuracy_Score)

0.82616


In [10]:
from tkinter import * 
window=Tk(className="Sentiment Analysis")
window.geometry('420x234')

Label(window, text='Test with your own text').grid(row=0) 
t1= Text(window, width=25,height=10,highlightcolor="blue",highlightthickness=3,relief=RAISED)
t1.grid(row=1)

Label(window, text='Results').grid(row=0,column=1)
t2= Text(window, width=25,height=10,highlightcolor="blue",highlightthickness=3,relief=RAISED)
t2.grid(row=1,column=1)

def retrieve_input():
    inputValue=t1.get("1.0","end-1c")
    if inputValue:
        x=CNB.predict(cv.transform([inputValue.lower()]))
        t2.delete('1.0', END)
        t2.insert(INSERT, x[0])
    else:
        t2.delete('1.0', END)

b= Button(window, width=20,height=2,text="Classify Text", bg="blue", fg="white",cursor="circle",command=lambda:                 retrieve_input(),relief=RAISED)
b.grid(row=2,column=0)

window.mainloop()